In [74]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.transforms
import matplotlib.cm
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
wind_angle = -np.pi/4
wind_mag = 1.
fly_mag = 1.5
flux_rad = 10.

num_flies = 15000
heading_angles = np.linspace(4*360./num_flies,4*360,num_flies)*np.pi/180

In [4]:
hit_angles = np.arctan((wind_mag*np.sin(wind_angle-heading_angles))/fly_mag)+heading_angles
cmap = matplotlib.cm.get_cmap('bwr')
colors = cmap(np.linspace(1./num_flies,1,num_flies))

In [5]:
from ipywidgets import interactive

In [75]:
def f(w=0.5,theta_d=np.pi/6.):
    n = 10000
    theta_0 = np.arctan(-1.)
    r_1 = 1.5
    thetas = np.linspace(360./n,360,n)*np.pi/180
    output = (np.arctan(w*np.sin(theta_0-thetas)/r_1)+thetas)%(2*np.pi)
    plt.figure(1,figsize=(16,8))
    gs = matplotlib.gridspec.GridSpec(2,4)
    plt.subplot(gs[0,0]);plt.title('Final vs. Intended Angle')
    plt.plot(thetas,output,'o',markersize=0.4)
    plt.xlim([0,2*np.pi]);plt.ylim([0,2*np.pi])
    plt.xticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
    plt.yticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
    
    w_vec = np.array([w*np.cos(theta_0),w*np.sin(theta_0)])
    r_vec = np.array([r_1*np.cos(theta_d),r_1*np.sin(theta_d)])
    wind_par_vec = (np.dot(w_vec,r_vec))/(r_1**2)*r_vec
    w_perp_vec = w_vec - wind_par_vec
    heading_final_vec = r_vec+w_perp_vec
#     final_angle_demo = np.arctan(heading_final_vec[1]/heading_final_vec[0])
    final_angle_demo = (np.arctan(w*np.sin(theta_0-theta_d)/r_1)+theta_d)%(2*np.pi)
    
    input_demo=plt.plot([theta_d,theta_d],[0,final_angle_demo%(2*np.pi)],'r',label='intended angle')
    output_demo=plt.plot([0,theta_d],[final_angle_demo%(2*np.pi),final_angle_demo%(2*np.pi)],'g',label='final angle',)
    plt.legend(bbox_to_anchor=(0.6,0.9))
    
    plt.subplot(gs[1,0]);plt.title('Derivative of Final vs. Intended Angle',fontsize=10)
    deriv = np.diff(output)[np.abs(np.diff(output))<1]
    plt.plot(thetas[:-2],deriv/(np.max(deriv))*2*np.pi,color='b')
    plt.plot(thetas,np.zeros_like(thetas),color='y')
    plt.xticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
    
    ax=plt.subplot(gs[0,1],projection='polar') 
    n,bins,_ = plt.hist(output%(2*np.pi),bins=500)
    cum,bins,_ = plt.hist(output%(2*np.pi),bins=500,cumulative=True)
    cum = cum/len(output)
    ax.cla()
    plt.plot(bins,np.concatenate((n,[n[0]])))
    ax.set_yticks([])
    _,ymax = ax.get_ylim()
    plt.plot([final_angle_demo,0],[ymax,0],'g');plt.title('Final Angle Histogram',x=0.5,y=1.1)
    plt.plot([theta_d,0],[ymax,0],'r');
    plt.xticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
    
    ax = plt.subplot(gs[1,1]);plt.title('Final Angle CDF')
    plt.plot(bins[:-1],cum)
    plt.xlim([0,2*np.pi])
    _,ymax = ax.get_ylim()
    
    plt.plot([final_angle_demo,final_angle_demo],[0,ymax],'g',label='final angle')
    plt.legend(bbox_to_anchor=(.5,.2))
    
    
    plt.subplot(gs[:,2:4]);plt.title('Vector Summation for Theta_d')
    original_heading = plt.arrow(0,0,r_1*np.cos(theta_d),r_1*np.sin(theta_d), head_width=0.05, head_length=0.1, fc='r', ec='r',
                                length_includes_head=True)
   
    wind_vector = plt.arrow(*r_vec,*w_vec,
                            head_width=0.05, head_length=0.1, fc='b', ec='b',length_includes_head=True)
#     wind_par = plt.arrow(*r_vec,*wind_par_vec, 
#                             head_width=0.05, head_length=0.1, fc='orange', ec='orange')
    wind_perp = plt.arrow(*r_vec,*w_perp_vec,
                            head_width=0.05, head_length=0.1, fc='dodgerblue', ec='dodgerblue',length_includes_head=True)
    wind_par = plt.arrow(*r_vec,*wind_par_vec,
                            head_width=0.05, head_length=0.1, fc='dodgerblue', ec='dodgerblue',length_includes_head=True)
    heading_final = plt.arrow(0,0,*heading_final_vec,
                             head_width=0.05, head_length=0.1, fc='g', ec='g',length_includes_head=True)
    
    plt.ylim([-2,3]);plt.xlim([-2,3])
    plt.legend([original_heading,wind_vector,wind_perp,wind_par,heading_final],
                   ['intended velocity','wind vector','wind perp','wind par','final velocity'],
                    bbox_to_anchor=(1.4,0.5))
    
    plt.show()
interactive_plot = interactive(f, w=(0, 6.0,0.05),theta_d=(0,2*np.pi,0.05))
output = interactive_plot.children[-1]
output.layout.height = '700px'
interactive_plot

interactive(children=(FloatSlider(value=0.5, description='w', max=6.0, step=0.05), FloatSlider(value=0.5235987…

In [7]:
def gs_from_wind(windspeed):
    #Given the magnitude and sign of the wind in the direction 
    #of the intended velocity (signed wind_par),
    #return the adjusted magnitude of the
    #fly's intended velocity
    if -0.8<=windspeed<=4.:
        return 1.6
    elif windspeed<-0.8:
        return windspeed+2.4
    elif windspeed>4.:
        return windspeed-2.4
    
def signed_wind_par_cartesian(wind_par,original_heading):
    #version 1: inputs are in cartesian coordinates
    #Given the component of wind parallel to the intended velocity,
    #(wind_par vector) and the intended velocity vector,
    #return the signed magnitude of the parallel wind 
    #(+ sign = same direction as intended velocity, - sign = opposite)
    #(prep function for gs_from_wind)
    sign = np.sign(np.dot(wind_par,original_heading))
    return sign*np.sqrt(original_heading.dot(original_heading))
        
def signed_wind_par_polar(theta_0,theta,w):
    #Similar to above except start with wind and intended velocity angle/magnitude
    #theta_0: wind angle
    #theta: intended velocity angle
    #w: wind magnitude
    return w*np.cos(theta_0-theta)
def pp_angle_map_array(theta_0,thetas,r_1s,w):
    wind_par_mags = signed_wind_par_polar(theta_0,thetas,w)
    thetas_adjusted = np.copy(thetas)
    r_1s[wind_par_mags<-0.8] = wind_par_mags[wind_par_mags<-0.8]+2.4
    r_1s[wind_par_mags>4.] = wind_par_mags[wind_par_mags>4.]-2.4
    sign_change_inds = (r_1s<0.)
    thetas_adjusted[sign_change_inds] = (thetas[sign_change_inds]+np.pi)%(2*np.pi)
    return (np.arctan(w*np.sin(theta_0-thetas_adjusted)/np.abs(r_1s))+thetas_adjusted)%(2*np.pi)

def pp_angle_map(theta_0,theta,w):
    wind_par_mag = signed_wind_par_polar(theta_0,theta,w)
    r_1 = gs_from_wind(wind_par_mag)
    if r_1<0:
        theta = (theta+np.pi)%(2*np.pi)
    return (np.arctan(w*np.sin(theta_0-theta)/np.abs(r_1))+theta)%(2*np.pi)



In [90]:
def f(w=0.9,theta_d=1.5):#1.*np.pi/4.):
    n = 10000 
    theta_0 = np.arctan(-1.)
#     theta_0 = 13*np.pi/8.

    r_1 = 1.6
    
    
    thetas = np.linspace(360./n,360,n)*np.pi/180

    #Adjust w for the fly's airspeed limits
    #only starts being relevant when w>0.8
    
    r_1s = 1.6*np.ones_like(thetas)
    
    saturation = (w>0.8)
    over_4 = (w>4.)
    
    if saturation:
        wind_par_mags = signed_wind_par_polar(theta_0,thetas,w)
        inds_limit_affected = (wind_par_mags<-0.8) | (wind_par_mags>4.)
        thetas_limit_affected = thetas[inds_limit_affected]
        if over_4:
            theta_min_1,theta_max_1 = min(thetas[wind_par_mags<-0.8]),max(thetas[wind_par_mags<-0.8])
            theta_min_2,theta_max_2 = min(thetas[wind_par_mags>4.]),max(thetas[wind_par_mags>4.])
        wind_par_mags_limit_affected = wind_par_mags[inds_limit_affected]
        
    output = pp_angle_map_array(theta_0,thetas,r_1s,w)
    if saturation:
        output_limit_affected = output[inds_limit_affected]
        two_patches =  (np.sum(np.diff(output)>6.)>0.) #Case where there's a circular jump
        if two_patches:
            jump_inds = np.where(np.abs(np.diff(output))>2.)[0]
            jumpless_affected = np.copy(output)
            jumpless_affected[jump_inds[0]+1:jump_inds[1]+1] = jumpless_affected[jump_inds[0]+1:jump_inds[1]+1] - np.sign(np.diff(output)[jump_inds[0]])*2*np.pi 
            output_affected_edges = np.max(jumpless_affected)%(2*np.pi), \
                np.min(jumpless_affected)%(2*np.pi)
 
            
        else:
            
            output_affected_edges = np.min(output_limit_affected)%(2*np.pi), \
                np.max(output_limit_affected)%(2*np.pi)
                  
    plt.figure(1,figsize=(15,15))
    gs = matplotlib.gridspec.GridSpec(4,4)
    
    plt.subplot(gs[0,0]);plt.title('Final vs. Intended Angle')
    ax=plt.plot(thetas,output,'o',markersize=0.4)
    plt.xlim([0,2*np.pi]);plt.ylim([0,2*np.pi])
    plt.xticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
    plt.yticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
    if saturation:
        if over_4:  
            plt.axvspan(theta_min_1,theta_max_1,
               alpha=0.2,color='k')
            plt.axvspan(theta_min_2,theta_max_2,
               alpha=0.2,color='k')
        else:
            plt.axvspan(np.min(thetas_limit_affected),
                        np.max(thetas_limit_affected),
                           alpha=0.2,color='k')
    
    w_vec = np.array([w*np.cos(theta_0),w*np.sin(theta_0)])
    wind_par_d = signed_wind_par_polar(theta_0,theta_d,w)
    r_1_d = gs_from_wind(wind_par_d)
    r_vec = np.array([r_1_d*np.cos(theta_d),r_1_d*np.sin(theta_d)])
    wind_par_vec = (np.dot(w_vec,r_vec))/(r_1_d**2)*r_vec
    w_perp_vec = w_vec - wind_par_vec
    heading_final_vec = r_vec+w_perp_vec
#     final_angle_demo = np.arctan(heading_final_vec[1]/heading_final_vec[0])
    final_angle_demo = pp_angle_map(theta_0,theta_d,w)
    
    input_demo=plt.plot([theta_d,theta_d],[0,final_angle_demo%(2*np.pi)],'r',label='intended angle')
    output_demo=plt.plot([0,theta_d],[final_angle_demo%(2*np.pi),final_angle_demo%(2*np.pi)],'g',label='final angle',)
    plt.legend(bbox_to_anchor=(-0.2,0.2))
    
    plt.subplot(gs[1,0]);plt.title('Derivative of Final vs. Intended Angle',fontsize=10)
    deriv = np.diff(output)[np.abs(np.diff(output))<1]
    plt.plot(thetas[:-1-np.sum(np.abs(np.diff(output))>1)],deriv,'o',markersize=0.4)
    plt.plot(thetas,np.zeros_like(thetas),color='y')
    plt.xticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
  #  plt.yticks([])
    if saturation:
        if over_4:  
            patch=plt.axvspan(theta_min_1,theta_max_1,
               alpha=0.2,color='k')
            plt.axvspan(theta_min_2,theta_max_2,
               alpha=0.2,color='k')
        else:
            patch=plt.axvspan(np.min(thetas_limit_affected),
                        np.max(thetas_limit_affected),
                           alpha=0.2,color='k')
        plt.legend([patch],['Values Affected by Airspeed Saturation'],bbox_to_anchor=(3.6,-0.5))
    
    ax=plt.subplot(gs[0,1],projection='polar') 
    n,bins,_ = plt.hist(output%(2*np.pi),bins=500)
    ax.cla()
    plt.plot(bins,np.concatenate((n,[n[0]])))
    ax.set_yticks([])
    _,ymax = ax.get_ylim()
#     plt.plot([final_angle_demo,0],[ymax,0],'g');
    plt.arrow(0,0,0,ymax/2,color='b',head_width=0.1,head_length=4,
                transform=matplotlib.transforms.Affine2D().translate(theta_0, 0) + ax.transData);
    plt.title('Final Angle Histogram',x=0.5,y=1.1)
    plt.xticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
#     if saturation:
#         if two_patches:
#             plt.fill_between(np.linspace(0,output_affected_edges[0]),0,ymax,alpha=0.2,color='k')
#             plt.fill_between(np.linspace(output_affected_edges[1],2*np.pi),0,ymax,alpha=0.2,color='k')
#         else:
#             ax.fill_between(np.linspace(*output_affected_edges),0,ymax,alpha=0.2,color='k')
        
    ax = plt.subplot(gs[1,1])
    cum,bins,_ = plt.hist(output%(2*np.pi),bins=500,cumulative=True)
    cum = cum/len(output)
    ax.cla()
    plt.plot(bins[:-1],cum);plt.title('Final Angle CDF')#,x=0.5,y=1.1)
    plt.xlim([0,2*np.pi])
    plt.xticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
    _,ymax = ax.get_ylim()
    
    
#     ax = plt.subplot(gs[1,1]);plt.title('Final Angle Histogram')
#     plt.plot(bins,np.concatenate((n,[n[0]])))
#     plt.xticks(np.linspace(np.pi/2,2*np.pi,4),('$\pi/2$','$\pi$','$3\pi/2$','$2\pi$'))
#     ax.set_yticks([])
#     plt.xlim([0,2*np.pi])
#     _,ymax = ax.get_ylim()
#     if saturation:
#         if two_patches:
#             plt.axvspan(0,output_affected_edges[0],alpha=0.2,color='k')
#             plt.axvspan(output_affected_edges[1],2*np.pi,alpha=0.2,color='k')
#         else:
#             plt.axvspan(*output_affected_edges,
#                            alpha=0.2,color='k')

    plt.plot([final_angle_demo,final_angle_demo],[0,ymax],'g',label='final angle')
    plt.legend(bbox_to_anchor=(0.5,0.8))
    
    
    plt.subplot(gs[0:2,2:4]);plt.title('Vector Summation for Theta_d')
    original_heading = plt.arrow(0,0,r_1_d*np.cos(theta_d),r_1_d*np.sin(theta_d), head_width=0.05, head_length=0.1, fc='r', ec='r',
                                length_includes_head=True)
   
    wind_vector = plt.arrow(*r_vec,*w_vec,
                            head_width=0.05, head_length=0.1, fc='b', ec='b',length_includes_head=True)
#     wind_par = plt.arrow(*r_vec,*wind_par_vec, 
#                             head_width=0.05, head_length=0.1, fc='orange', ec='orange')
    wind_perp = plt.arrow(*r_vec,*w_perp_vec,
                            head_width=0.05, head_length=0.1, fc='dodgerblue', ec='dodgerblue',length_includes_head=True)
    par_adjustment_vec = -0.05*(w_perp_vec/(np.sqrt(w_perp_vec.dot(w_perp_vec))))
    #to avoid directly superimposing the intended vector and the parallel wind vector, insert a small shift
    #to the parallel wind vector in the direction of perp wind vector
    wind_par = plt.arrow(*(r_vec+par_adjustment_vec),*(wind_par_vec),
                            head_width=0.05, head_length=0.1, fc='deepskyblue', ec='deepskyblue',length_includes_head=True)
    heading_final = plt.arrow(0,0,*heading_final_vec,
                             head_width=0.05, head_length=0.1, fc='g', ec='g',length_includes_head=True)
    
    plt.ylim([-2,4]);plt.xlim([-2,4])
    plt.legend([original_heading,wind_vector,wind_perp,wind_par,heading_final],
                   ['intended velocity','wind vector','wind perp','wind par','final velocity'],
                    bbox_to_anchor=(0.95,0.25))#bbox_to_anchor=(1.4,0.5))
    
    
    ax=plt.subplot(gs[2:3,0:2]);plt.title('Magnitude of Intended Velocity by Parallel Windspeed Magnitude')
    
    xmin=-4.;xmax=6.
    plt.xlim([xmin,xmax])
    plt.plot([-4,-0.8],[-1.6,1.6],color='indianred',label='groundspeed (intended)')
    plt.plot([-0.8,4.],[1.6,1.6],color='indianred')
    plt.plot([4.,6.],[1.6,3.6],color='indianred')
    
    plt.plot([xmin,-0.8],[2.4,2.4],color='orange',label='airspeed')
    plt.plot([-0.8,4.],[2.4,-2.4],color='orange')
    plt.plot([4.,xmax],[-2.4,-2.4],color='orange')
    
    
#     plt.plot([wind_par_d,wind_par_d],[-3.,r_1_d],'dodgerblue')
#     plt.plot([ax.get_xlim()[0],wind_par_d],[r_1_d,r_1_d],'r')
    plt.plot([ax.get_xlim()[0],ax.get_xlim()[1]],[0,0],'0.1')
    if saturation:
        if over_4:
            plt.axvspan(np.min(wind_par_mags_limit_affected),-0.8,
                       alpha=0.2,color='k')
            patch = plt.axvspan(4.,np.max(wind_par_mags_limit_affected),
                        alpha=0.2,color='k')
            
#         else:
#             patch = plt.axvspan(np.min(wind_par_mags_limit_affected),
#                         np.max(wind_par_mags_limit_affected),
#                            alpha=0.2,color='k')
    
    plt.legend(bbox_to_anchor=(1.2,0.4))
#     plt.legend(bbox_to_anchor=(0.5,0.01))
    plt.xlabel('Parallel Windspeed')
    plt.savefig('perp_slip_analysis_viz_faculty_talk.png',dpi=500,format='png')
    
    plt.show()
    

In [91]:
interactive_plot = interactive(f, w=(0, 6.0,0.05),theta_d=(0,2*np.pi,0.1),continuous_update=False)
output = interactive_plot.children[-1]
output.layout.height = '1000px'
interactive_plot

interactive(children=(FloatSlider(value=0.9, description='w', max=6.0, step=0.05), FloatSlider(value=1.5, desc…